# Evaluate land model output from perturbed parameter ensemble

This script evaluates model output from a set of ensemble members in a perturbed parameter experiment. It identifies the best-performing ensemble members

## Import modules

In [9]:
import sys
#Path to the esm_tools.py script
sys.path.append('/glade/u/home/adamhb/Earth-System-Model-Tools/process_output')
import os
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functools
import netCDF4 as nc4
import importlib
import esm_tools
import esm_viz
importlib.reload(esm_tools)
importlib.reload(esm_viz)
import math
pd.set_option('display.max_rows', 500) 

## User-defined parameters

In [10]:
case_name = 'ca_5pfts_20cases_4320inst_101223_02_-17e2acb6a_FATES-031f28ff'

# Subdirectory where the parameter file for each ensemble member is stored
param_sub_dir="ca_5pfts_20cases_4320inst_101223_02"

# File name of the file that stores the parameter ranges for the ensemble
param_range_file_name = 'param_ranges_100223.csv'

# How many years of data to average over for the structure variables
last_n_years=1

# Just testing script?
test=False

# For tree stem density
dbh_min = 10

# Calculate variables that require loading much more data (e.g. fire regime?)
decadal_scale_metrics=False

# How many years of data to average over for the structure variables
decadal_n_years=30

visualize = False

# Optional
case_path = None
manual_case_path = None

## Define paths and script parameters

In [11]:
pft_names = ["pine","cedar","fir","shrub","oak"]

# Benchmarking metrics
my_metrics = ["BA","AGB","TreeStemD","ResproutD_oak","ResproutD_shrub","ShannonE","NPP","FailedPFTs",
              "Pct_shrub_cover_canopy","Pct_shrub_cover",
              "Combustible_fuel"]

if decadal_scale_metrics == True:
    my_metrics.extend(["Burned_area","Pct_high_severity_1700","Pct_high_severity_3500"])

# Path where case output lives
case_output_root = '/glade/scratch/adamhb/archive'

# Path to ensemble params
params_root = '/glade/u/home/adamhb/ahb_params/fates_api_25/ensembles'

# Path to put any processed output
processed_output_root = '/glade/scratch/adamhb/processed_output'

# Path to param range files
param_range_root = '/glade/u/home/adamhb/california-fates/parameter_ranges/param_range_archive'

output_path_for_case = os.path.join(processed_output_root,case_name)
esm_tools.create_directory(output_path_for_case)

print("Calculating the following variables:",my_metrics)

Directory '/glade/scratch/adamhb/processed_output/ca_5pfts_20cases_4320inst_101223_02_-17e2acb6a_FATES-031f28ff' created successfully!
Calculating the following variables: ['BA', 'AGB', 'TreeStemD', 'ResproutD_oak', 'ResproutD_shrub', 'ShannonE', 'NPP', 'FailedPFTs', 'Pct_shrub_cover_canopy', 'Pct_shrub_cover', 'Combustible_fuel']


## Variables to import

In [12]:
# Keep first two no matter what. They are needed to unravel multi-plexed dimensions
fields = ['FATES_SEED_PROD_USTORY_SZ','FATES_VEGC_AP','FATES_BURNFRAC',
          'FATES_NPLANT_PF','FATES_NPLANT_SZPF','FATES_NPLANT_RESPROUT_PF','FATES_FIRE_INTENSITY_BURNFRAC','FATES_IGNITIONS',
          'FATES_MORTALITY_FIRE_SZPF','FATES_BASALAREA_SZPF','FATES_CANOPYCROWNAREA_APPF','FATES_CANOPYCROWNAREA_PF','FATES_CROWNAREA_PF',
          'FATES_CROWNAREA_APPF','FATES_FUEL_AMOUNT_APFC','FATES_NPLANT_SZPF','FATES_FUEL_AMOUNT_APFC',
          'FATES_PATCHAREA_AP','FATES_CROWNAREA_PF','FATES_VEGC_ABOVEGROUND','FATES_NPP_PF']

## Benchmarking functions

In [13]:
def setup_benchmarking_data_structure(metrics,parameters,pft_names):
    
    metrics_out = metrics.copy()
    
    # add pft-specific vars
    pft_specific_ba_metrics = ["BA_" + pft for pft in pft_names]  
    metrics_out.extend(pft_specific_ba_metrics)
    
    # add inst tag
    metrics_out.append("inst")    
    metrics_out.extend(parameters)
    
    benchmarking_dict = {}
    for i in metrics_out:
        benchmarking_dict[i] = []
    return benchmarking_dict

def get_benchmarks(case_name,metrics,last_n_years,param_sub_dir,param_range_file_name,
                   test = False, pft_names = np.array(["pine","cedar","fir","shrub","oak"]),
                   pft_colors = ['gold','darkorange','darkolivegreen','brown','springgreen'],
                   param_range_root = param_range_root,
                   params_root = params_root,
                   manual_case_path = None, decadal_scale_metrics = False, decadal_n_years = 50):
      
    print("Case:",case_name)
    
    # 1. Get info about the case
    if manual_case_path != None:
        full_case_path = manual_case_path
    
    else:
        full_case_path = esm_tools.get_path_to_sim(case_name,case_output_root)
    
    inst_tags = esm_tools.get_unique_inst_tags(full_case_path)
    
    if test == True:
        inst_tags = inst_tags[:3]
    
    n_inst = len(inst_tags)
    print("ninst:",n_inst)
    
    # 2. Set up the benchmarking data structure
    perturbed_params_df = pd.read_csv(os.path.join(param_range_root,param_range_file_name))
    perturbed_params = []
    for i in range(len(perturbed_params_df)):
        perturbed_params.append(perturbed_params_df['param'][i] + "_" + str(perturbed_params_df['pft'][i]))
    
    bench_dict = setup_benchmarking_data_structure(metrics,perturbed_params,pft_names)  
                                 
    # 3. Add param values to the data structure
    for inst in inst_tags:
        param_file_path = esm_tools.get_parameter_file_of_inst(params_root,param_sub_dir,inst)
        #print(param_file_path)
        for i in perturbed_params_df.index:
            
            d = perturbed_params_df.loc[i]
            param = d['param']
            pft_index = max(0,int(d['pft'] - 1))
            organ = d['organ']
            
            if (param == "fates_frag_maxdecomp") & (organ > 1):
                continue
                
            if math.isnan(organ):
                organ_index = None
            else:
                organ_index = int(organ - 1)
           
            bench_dict[perturbed_params[i]].append(esm_tools.extract_variable_from_netcdf_specify_organ(
                                                           param_file_path,param,pft_index,organ_index))
    
    # 4. Add the model output to the data structure
    for inst in inst_tags:
        
        print("Working on ensemble memeber",inst,"of",len(inst_tags),"members")
        
        # Import the model output data for one ensemble member
        inst_files_last_n_years = esm_tools.get_files_of_inst(full_case_path,
                                                 inst,
                                                 last_n_years)
        
        ds = esm_tools.multiple_netcdf_to_xarray(inst_files_last_n_years,fields)
        
        
        bench_dict['inst'].append(inst)
        
        ## Basal area [m2 ha-1] ##
        if "BA" in bench_dict.keys():
            
            ## Pft-specific BA
            pft_level_ba = esm_tools.get_pft_level_basal_area(ds)
            
            for i in range(len(pft_names)):
                pft_name = pft_names[i]
                bench_dict['BA_' + pft_name].append(pft_level_ba[i])
            
            ## Shannon equitability index (wrt BA) ##
            bench_dict['ShannonE'].append(esm_tools.shannon_equitability(pft_level_ba))
            
            ## Number of failed pfts ##
            bench_dict['FailedPFTs'].append(esm_tools.get_n_failed_pfts(pft_level_ba,ba_thresh=0.1))
            
            ## Total BA
            bench_dict['BA'].append(pft_level_ba.sum())
                  
        ## Stem density [N ha-1] ##
        if "TreeStemD" in bench_dict.keys():
            
            ## Total tree stem density
            bench_dict["TreeStemD"].append(esm_tools.get_total_stem_den(ds,trees_only=True,dbh_min=dbh_min))
        
        if "ResproutD_oak" in bench_dict.keys():
            bench_dict["ResproutD_oak"].append(esm_tools.get_resprout_stem_den(ds,4))
            
        if "ResproutD_shrub" in bench_dict.keys():
            bench_dict["ResproutD_shrub"].append(esm_tools.get_resprout_stem_den(ds,3))
        
        ## AGB [kg C m-2]
        if "AGB" in bench_dict.keys():
            bench_dict["AGB"].append(esm_tools.get_AGB(ds))
        
        ## Total NPP [kg C m-2]
        if "NPP" in bench_dict.keys():
            bench_dict["NPP"].append(esm_tools.get_total_npp(ds))
        
        ## Shrub canopy layer cover [m2 m-2]
        if "Pct_shrub_cover_canopy" in bench_dict.keys():
            bench_dict["Pct_shrub_cover_canopy"].append(esm_tools.get_pft_level_crown_area(ds,pft_index = 3))
            
        if "Pct_shrub_cover" in bench_dict.keys():    
            bench_dict["Pct_shrub_cover"].append(esm_tools.get_pft_level_crown_area(ds,pft_index = 3,canopy_area_only = False))
        
        ## Fuel Load
        if "Combustible_fuel" in bench_dict.keys():
            bench_dict["Combustible_fuel"].append(esm_tools.get_combustible_fuel(ds))
        
        if decadal_scale_metrics == True:
            inst_files_decadal = esm_tools.get_files_of_inst(full_case_path,
                                                 inst,
                                                 decadal_n_years)
            ds_decadal = esm_tools.multiple_netcdf_to_xarray(inst_files_decadal,fields)
            
            if "Burned_area" in bench_dict.keys():
                bench_dict["Burned_area"].append(esm_tools.get_mean_annual_burn_frac(ds_decadal))
                
            if "Pct_high_severity_1700" in bench_dict.keys():
                bench_dict["Pct_high_severity_1700"].append(esm_tools.get_PHS_FLI_thresh(ds_decadal,1700))
            
            if "Pct_high_severity_3500" in bench_dict.keys():
                bench_dict["Pct_high_severity_3500"].append(esm_tools.get_PHS_FLI_thresh(ds_decadal,3500))
            
        
    return bench_dict

## Apply to case

In [14]:
output_dict = get_benchmarks(case_name=case_name,
                              metrics = my_metrics,
                              last_n_years=last_n_years,
                              param_sub_dir=param_sub_dir,
                              param_range_file_name = param_range_file_name,
                              pft_names = pft_names,
                              test=test,
                              decadal_n_years=decadal_n_years,
                              decadal_scale_metrics=decadal_scale_metrics)#,
                              #manual_case_path = manual_case_path)
df = pd.DataFrame(output_dict)
esm_tools.store_output_csv(case_name,df,processed_output_root = processed_output_root)

Case: ca_5pfts_20cases_4320inst_101223_02_-17e2acb6a_FATES-031f28ff
ninst: 216
Working on ensemble memeber 0001 of 216 members
Working on ensemble memeber 0002 of 216 members
Working on ensemble memeber 0003 of 216 members
Working on ensemble memeber 0004 of 216 members
Working on ensemble memeber 0005 of 216 members
Working on ensemble memeber 0006 of 216 members
Working on ensemble memeber 0007 of 216 members
Working on ensemble memeber 0008 of 216 members
Working on ensemble memeber 0009 of 216 members
Working on ensemble memeber 0010 of 216 members
Working on ensemble memeber 0011 of 216 members
Working on ensemble memeber 0012 of 216 members
Working on ensemble memeber 0013 of 216 members
Working on ensemble memeber 0014 of 216 members
Working on ensemble memeber 0015 of 216 members
Working on ensemble memeber 0016 of 216 members
Working on ensemble memeber 0017 of 216 members
Working on ensemble memeber 0018 of 216 members
Working on ensemble memeber 0019 of 216 members
Working o

/glade/u/home/adamhb/Earth-System-Model-Tools/process_output/esm_tools.py:369: RuntimeWarning: divide by zero encountered in log
  h_i.append(p[i] * np.log(p[i]))
/glade/u/home/adamhb/Earth-System-Model-Tools/process_output/esm_tools.py:369: RuntimeWarning: invalid value encountered in float_scalars
  h_i.append(p[i] * np.log(p[i]))


Working on ensemble memeber 0089 of 216 members
Working on ensemble memeber 0090 of 216 members
Working on ensemble memeber 0091 of 216 members
Working on ensemble memeber 0092 of 216 members
Working on ensemble memeber 0093 of 216 members
Working on ensemble memeber 0094 of 216 members
Working on ensemble memeber 0095 of 216 members
Working on ensemble memeber 0096 of 216 members
Working on ensemble memeber 0097 of 216 members
Working on ensemble memeber 0098 of 216 members
Working on ensemble memeber 0099 of 216 members
Working on ensemble memeber 0100 of 216 members
Working on ensemble memeber 0101 of 216 members
Working on ensemble memeber 0102 of 216 members
Working on ensemble memeber 0103 of 216 members
Working on ensemble memeber 0104 of 216 members
Working on ensemble memeber 0105 of 216 members
Working on ensemble memeber 0106 of 216 members
Working on ensemble memeber 0107 of 216 members
Working on ensemble memeber 0108 of 216 members
Working on ensemble memeber 0109 of 216 

In [15]:
cols = list(df.columns)[:17]
df.sort_values("FailedPFTs",ascending=True)[cols]

,BA,AGB,TreeStemD,ResproutD_oak,ResproutD_shrub,ShannonE,NPP,FailedPFTs,Pct_shrub_cover_canopy,Pct_shrub_cover,Combustible_fuel,BA_pine,BA_cedar,BA_fir,BA_shrub,BA_oak,inst
113,26.286749,5.343412,618.887357,5.7909985,0.0,0.725270,0.773855,0,0.2561969,0.41269037,0.9016738,1.463966,15.330979,3.353898,5.348026e+00,0.789881,0114
61,17.591408,4.180392,240.508150,91.446365,0.0,0.654107,0.741108,0,0.13878751,0.27478305,1.176755,0.102047,3.160607,0.291298,3.808336e+00,10.229120,0062
122,17.176456,3.350533,566.527955,3.9262533,0.034739934,0.397407,0.539320,0,0.04524267,0.045784686,1.2510107,14.465503,1.077861,0.433022,8.437983e-01,0.356272,0123
130,43.928890,9.160060,162.281655,4.9915023,0.20352739,0.507081,0.837515,0,0.43931904,0.5780771,1.3675952,29.215729,0.711143,1.021034,1.270475e+01,0.276237,0131
197,22.953611,5.724550,382.332020,26.190216,0.0,0.538327,0.856668,0,0.0001483664,0.0074303686,0.4990573,10.883416,11.148107,0.134825,1.386854e-01,0.648578,0198
115,9.675728,1.882767,315.903127,51.76798,0.0,0.864083,0.425926,0,0.07596494,0.076853804,0.33113456,2.282976,2.309375,0.108063,1.512021e+00,3.463293,0116
20,15.271308,3.791176,307.381395,98.13263,0.0,0.782341,0.595313,0,0.08564919,0.16695327,0.46742105,0.414486,3.112776,0.614719,4.032365e+00,7.096962,0021
121,21.571098,2.550665,370.268822,14.67283,0.039617494,0.651514,0.626925,0,0.5754071,0.93200606,0.7960355,1.718607,1.180609,1.354752,1.473829e+01,2.578845,0122
17,26.023800,6.899467,207.635947,63.483322,0.0,0.776913,0.748294,0,4.685871e-05,0.01638557,0.4472251,8.668104,7.062406,0.804542,3.290413e-01,9.159708,0018
162,36.234081,9.073871,405.253358,0.22434944,0.0,0.451183,0.679698,0,8.620444e-06,0.011730403,0.6217644,28.083067,4.330201,3.480230,1.076991e-01,0.232882,0163


## Visualize

In [16]:
if visualize == True:
    col_selector = ["fates" in i for i in df.columns]
    perturbed_params = df.columns[col_selector]


    for p in perturbed_params:
        esm_viz.plot_multi_panel(df = df, x_col = p,
                                 y_cols = my_metrics, figsize=(12, 16),
                                 save_fig=True,
                                 output_path_for_case=output_path_for_case)